<a href="https://colab.research.google.com/github/stefymojica/MACC/blob/main/Modelo_clinico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agente para la optimizacion de recursos en un modelo clinico
## Matematicas aplicadas y ciencias de la computacion
## Estudiante : Stefany Paola Mojica Melo

## Contextualizacion

- Hay 3 agentes:
  - pacientes
  - medicos
  - triaja
- Hay un modelo
  - Clinica donde los agentes van a interactuar
- Los pacientes llegan y el triaje asigna el nivel de riesgo dependiendo de la situacion del paciente
- A la clinica llegan cada 25 minutos un nuevo paciente
- Los doctores se demoran 15 minutos para descubrir que le sucede a un paciennte -
- los pacientes llegan y el triaje les asigana un nivel de criticidad son 5 niveles.
  - nivel 1 - situacion de riesgo vital, requiere accion inmediata
  - nivel 2 - Emergencia. requiere atencion en un tiempo maximo de 30 minutos
  - nivel 3 - Urgencia. se debe atender en un tiempo no mayor a 30 minutos
  - nivel 4 - urgencia menor. Puede esperar hasta 120 minutos
  - nivel 5 - sin urgencia. puede esperar hasta 180 minutos

## Instalar librerias

In [11]:
%pip install mesa

In [9]:
import mesa

In [10]:
from mesa import Agent, Model
from mesa.space import MultiGrid
import random
import heapq

## Creacion de Agentes y Modelo

In [12]:
class Paciente(Agent):
  def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.prioridad = None
        self.atendido = False

  def step(self):
    pass

class Triaje(Agent):
  def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.ocupado = False

  def step(self):
    if not self.ocupado and len(self.model.pacientes_sin_triaje) > 0:
      paciente = self.model.pacientes_sin_triaje.pop(0)
      self.asignar_prioridad(paciente)

  def asignar_prioridad(self, paciente):
    r = random.random()
    if r < 0.05: paciente.prioridad = 1
    elif r < 0.20: paciente.prioridad = 2
    elif r < 0.50: paciente.prioridad = 3
    elif r < 0.80: paciente.prioridad = 4
    else: paciente.prioridad = 5

    heapq.heappush(self.model.cola_prioridad, (paciente.prioridad, paciente))

class Doctor(Agent):
  def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.ocupado = False
        self.paciente_actual = None

  def step(self):
    if not self.ocupado and len(self.model.cola_prioridad) > 0:
      _, paciente = heapq.heappop(self.model.cola_prioridad)
      self.atender(paciente)

    elif self.ocupado:
      self.ocupado = False
      self.paciente_actual = None

  def atender(self, paciente):
    self.ocupado = True
    self.paciente_actual = paciente
    paciente.atendido = True
    self.model.pacientes_atendidos += 1

class Clinica(Model):
  def __init__(self, n_doctores, tasa_llegada):
    self.pacientes_sin_triaje = []
    self.cola_prioridad = []
    self.pacientes_atendidos = 0
